In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# only if you are on google coalb not if you are on jupyter notebook
from google.colab import drive
drive.mount('/content/gdrive')

In [4]:
# only if you are on jupyter notebook not if on colab
import qgrid

/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt


In [4]:
# install package
!pip install oneibl

ERROR: Could not find a version that satisfies the requirement oneibl (from versions: none)
ERROR: No matching distribution found for oneibl


## Load Data


### via oneibl by client

In [5]:

from oneibl.onelight import ONE
one = ONE()
one.set_figshare_url("https://figshare.com/articles/steinmetz/9974357")


ModuleNotFoundError: ignored

In [ ]:
sessions = one.search(['spikes'])


### via google drive directly

In [5]:
def load_session(folder):
  """
  load all files from given folder to dict
  return dict with key as filenames
  """
  from os import listdir
  from os.path import isfile, join
  files = [f for f in listdir(folder) if isfile(join(folder, f))]
  names = ['_'.join(f.split('.')[:-1]) for f in files]
  session = dict()
  for (file_, name_) in zip(files, names):
    if file_.split('.')[-1] == 'npy':
      session[name_] = np.load(folder+'/'+file_)
    if file_.split('.')[-1] == 'tsv':
      session[name_] = pd.read_table(folder+'/'+file_)
  return session

# def load_session(folder, sessions):
#   """
#   load all files in each given session and add to session dict
#   returs a dict of sessions, with each session a dict of files
#   """
#   sessions_data = dict()
#   for session in sessions:
#     current_session = dict()
#     sessions[session] = 

In [95]:
# example google colab load session
folder = "/content/gdrive/My Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
sessions = [folder.split('/')[-1] for folder in subfolders]

# load first session
session = load_session(subfolders[0])

In [8]:
# example windows load session
folder = "C:/Users/User/Google Drive/3 Projekte/Neuromatch Summer School/Projekt/steinmetz_dataset"
subfolders = [ f.path for f in os.scandir(folder) if f.is_dir() ]
sessions = [folder.split('/')[-1] for folder in subfolders]

# load first session
session = load_session(subfolders[0])

In [9]:
for key in session.keys():
  print(key)

channels_brainLocation
channels_probe
channels_rawRow
channels_site
channels_sitePositions
clusters_depths
clusters_originalIDs
clusters_peakChannel
clusters_probes
clusters_templateWaveformChans
clusters_templateWaveforms
clusters_waveformDuration
clusters__phy_annotation
Cori_2016-12-14_M2_g0_t0_imec_lf_timestamps
Cori_2016-12-14_V1_g0_t0_imec_lf_timestamps
eye_area
eye_timestamps
eye_xyPos
face_motionEnergy
face_timestamps
lickPiezo_raw
lickPiezo_timestamps
licks_times
passiveBeeps_times
passiveValveClick_times
passiveVisual_contrastLeft
passiveVisual_contrastRight
passiveVisual_times
passiveWhiteNoise_times
probes_description
probes_insertion
probes_rawFilename
probes_sitePositions
sparseNoise_positions
sparseNoise_times
spikes_amps
spikes_clusters
spikes_depths
spikes_times
spontaneous_intervals
trials_feedbackType
trials_feedback_times
trials_goCue_times
trials_included
trials_intervals
trials_repNum
trials_response_choice
trials_response_times
trials_visualStim_contrastLeft
tria

### cleaning up spikes

In [26]:
#spikes = pd.DataFrame()
spikes_df = pd.DataFrame( { 'cluster':session['spikes_clusters'][:,0], 'spike_times': session['spikes_times'][:,0] }, ) 
qgrid.show_grid(spikes_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
#clusters = spikes.spikes_df.groupby('cluster')
spk = pd.DataFrame( {'spikes':np.zeros(test2.shape[0], dtype=object)}, index=test2.index )
for group, frame in spikes.spikes_df.groupby('cluster'):
    spk['spikes'][group] = frame['spike_times'].values
spk

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [24]:
spikes_df = pd.DataFrame( {'cluster':session['spikes_clusters'][:,0]})